# Dashboard 

*Disclaimer: Das Dashboard wurde zunächst mit Vise-Coding aufgebaut: Es wurde mit ChatGPT konzipiert und in allen Details manuell optimiert.*

Zum Abschluss des Projekts soll ein interaktives Dashboard mit Streamlit erstellt werden.

in einem ersten Schritt wird ein MVP (Minimal Viable Product) entworfen, um die Funktionalitäten zu entwickeln und zu testen. Das MVP enthält nur Daten aus drei Städten: Hamburg, Berlin und Karlsruhe.

Der Datensatz für das MVP ist gespeichert als `test_dashboard_air_quality.csv`.

Note to self: Run app from VS Code terminal with "uv run streamlit run app.py"



# 🧭 Plan für das Streamlit-Dashboard zur Luftqualität

## ✅ 1. Basisfunktionen (bereits umgesetzt oder fast fertig):

- Dropdown zur Stadtauswahl (Stadt A & Stadt B)
- Dropdown zur Schadstoffauswahl (z. B. PM2.5, NO₂, SO₂, O₃, CO)
- Liniendiagramm mit dem gewählten Schadstoff über die Zeit
- Vergleich zweier Städte im gemeinsamen Diagramm
- Achsen-Labels und Titel verbessern (PM2.5 statt „Pm25“ usw.)

## 🧩 2. Nächste Ausbaustufe (geplant):
a) Zusätzliche Filtermöglichkeiten

- Auswahl eines Jahres oder Monats (Slider oder Selectbox)
- Filter nach Jahreszeit (z. B. Frühling, Sommer...)

b) Statistische Kennzahlen

- Durchschnitt, Minimum, Maximum des gewählten Schadstoffs für jede Stadt
- Darstellung unterhalb des Plots oder in einer kleinen Infobox

c) Darstellung Wetterdaten

- Auswahl eines Wetterparameters (Tavg, Humidity, etc.)
- Zweiter Plot oder zusätzliche Linie im selben Plot (Sekundärachse)

## 🖼 3. Späterer Ausbau (optional):
a) Interaktive Visualisierungen mit Plotly

- Hover-Effekte, dynamisches Zoomen, bessere Tooltips

b) Karte mit Clusterfarben

- z. B. Städte auf einer Karte mit Farben nach Cluster oder Mittelwert PM2.5

c) Infoboxen / Erläuterungen

- Kleine Textboxen mit verständlichen Erklärungen zur Interpretation der Daten
- Z. B. „PM2.5 ist Feinstaub unter 2.5 Mikrometer…“

d) Dashboard mit Datums- und Städteauswahl

- Auswahl eines Datums → Anzeige der Messwerte für alle Variablen
- Interaktive Heatmap oder Vergleichstabelle

## 📚 Inhaltsverzeichnis 
(Diese Art von Inhaltsverzeichnis mit Link funktioniert leider in Notebooks nicht, weil die as JSON gespeichert werden und nicht als HTML...)

- [0. Datensatz laden](#0-datensatz-laden)
- [1. Dataframe für MVP vorbereiten]


# 0. Datensatz laden

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("data/cleaned_air_quality_data_2025-03-27.csv")
df.head()

# 1. Dataframe für MVP vorbereiten

Für das MVP sollten drei Städte gewählt werden, die möglichst vollständige Messdaten für alle relevanten Variablen übermittelt haben, damit die Funktionalitäten gut getestet werden können. Auch sollte die Anzahl an Datenpunkten pro Stadt ausreichend sein und den geamten relevanten Zeitraum abdecken.

In [ ]:
# Optional: nur Spalten betrachten, die im Dashboard relevant sind
relevante_spalten = ["City", "Year", "Month", "Day", "Pm25", "Co", "No2", "So2", "O3", "Tavg", "Humidity"]
df = df[relevante_spalten]

# NaN-Anzahl je Stadt berechnen (über alle relevanten Spalten hinweg)
stadt_nan = df.groupby("City").apply(lambda x: x.isna().sum().sum())

# Alternativ: Anteil an NaN-Werten je Stadt
stadt_nan_anteil = df.groupby("City").apply(lambda x: x.isna().mean().mean())

# Sortieren: Städte mit den wenigsten fehlenden Werten zuerst
stadt_nan = stadt_nan.sort_values()
stadt_nan_anteil = stadt_nan_anteil.sort_values()

# Die Top 10 anzeigen
print("Städte mit absolut wenigsten NaNs:")
print(stadt_nan.head(10))

print("\nStädte mit dem geringsten Anteil an NaNs:")
print(stadt_nan_anteil.head(10))


In [ ]:
# Liste der drei gewünschten Städte
städte = ["Delhi", "Tokyo", "Osaka"]

# Relevante Spalten definieren
relevante_spalten = ["Pm25", "Co", "No2", "So2", "O3", "Tavg", "Humidity"]

# DataFrame auf die drei Städte und die relevanten Spalten beschränken
df_subset = df[df["City"].isin(städte)][["City"] + relevante_spalten]

# Fehlende Werte je Stadt und Variable zählen
fehlende_werte = df_subset.groupby("City").apply(lambda x: x[relevante_spalten].isna().sum())

# Alternativ: Anteil an NaN-Werten je Stadt und Variable
anteil_nan = df_subset.groupby("City").apply(lambda x: x[relevante_spalten].isna().mean())

# Ergebnisse anzeigen
print("Anzahl fehlender Werte:")
print(fehlende_werte)

print("\nAnteil fehlender Werte:")
print(anteil_nan.round(2))


In [ ]:
# Test-DataFrame für das Dashboard erstellen
df_mvp = df[df["City"].isin(["Delhi", "Tokyo", "Osaka"])].copy()

# Relevante Spalten wählen
columns = ["Year", "Month", "Day", "City", "Pm25", "Co", "No2", "So2", "O3", "Tavg", "Humidity"]
df_mvp = df_mvp[columns]

# Fürs MVP: nur Zeilen ohne NaNs behalten
df_mvp.dropna(inplace=True)

# In Datei speichern
df_mvp.to_csv("data/test_dashboard_air_quality.csv", index=False)


# Zeitreihenzerlegung für die Teststädte, analog zu NB 6

In [ ]:
# Create date column
df_mvp["Date"] = pd.to_datetime(df_mvp[["Year", "Month", "Day"]])
# Set date as index
df_mvp.set_index("Date", inplace=True)

# Plotting
# plt.figure(figsize=(14, 7))
# plt.plot(df_mvp[df_mvp["City"] == "Delhi"]["Pm25"], label="Delhi Pm25")
# plt.plot(df_mvp[df_mvp["City"] == "Tokyo"]["Pm25"], label="Tokyo Pm25")
# plt.plot(df_mvp[df_mvp["City"] == "Osaka"]["Pm25"], label="Osaka Pm25")
# plt.title("Pm25 Levels in Delhi, Tokyo, and Osaka")
# plt.xlabel("Date")
# plt.ylabel("Pm25 Levels")
# plt.legend()
# plt.grid();

In [ ]:
# Osaka
df_osaka = df_mvp[df_mvp["City"] == "Osaka"].copy()
df_osaka.head()


In [ ]:
# Tokyo
df_tokyo = df_mvp[df_mvp["City"] == "Tokyo"].copy()
df_tokyo.shape

In [ ]:
# Delhi
df_delhi = df_mvp[df_mvp["City"] == "Delhi"].copy()
df_delhi.shape

In [ ]:

from statsmodels.tsa.seasonal import seasonal_decompose

# Filtern: Nur Daten von 2015 bis 2024 behalten
df_tokyo_filtered = df_tokyo[(df_tokyo.index.year >= 2015) & (df_tokyo.index.year <= 2024)]

# Überprüfen, welche Jahre jetzt enthalten sind
print("Enthaltene Jahre:", sorted(df_tokyo_filtered.index.year.unique()))

# Aggregiere die PM2.5-Daten auf monatlicher Basis (mittlere Werte)
df_tokyo_monthly = df_tokyo_filtered['Pm25'].resample('M').mean().dropna()

# Zeitreihenzerlegung (additives Modell mit period=12 für monatliche Daten
result = seasonal_decompose(df_tokyo_monthly, model='additive', period=12)

# Visualisierung der Zerlegung
result.plot()

plt.suptitle('Zeitreihenzerlegung der monatlichen PM2.5-Daten in Tokyo (2015-2024)', fontsize=16)
plt.tight_layout();

In [ ]:
# Trend extrahieren und als DataFrame speichern
trend_tokyo = result.trend.dropna().reset_index()
trend_tokyo.columns = ["Datum", "Trend"]
trend_tokyo["City"] = "Tokyo"

In [ ]:
trend_tokyo

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Filtern: Nur Daten von 2015 bis 2024 behalten
df_osaka_filtered = df_osaka[(df_osaka.index.year >= 2015) & (df_osaka.index.year <= 2024)]

# Überprüfen, welche Jahre jetzt enthalten sind
print("Enthaltene Jahre:", sorted(df_osaka_filtered.index.year.unique()))

# Aggregiere die PM2.5-Daten auf monatlicher Basis (mittlere Werte)
df_osaka_monthly = df_osaka_filtered['Pm25'].resample('M').mean().dropna()

# Zeitreihenzerlegung (additives Modell mit period=12 für monatliche Daten
result = seasonal_decompose(df_osaka_monthly, model='additive', period=12)

# Visualisierung der Zerlegung
result.plot()

plt.suptitle('Zeitreihenzerlegung der monatlichen PM2.5-Daten in Osaka (2015-2024)', fontsize=16)
plt.tight_layout();

In [ ]:
# Trend extrahieren und als DataFrame speichern
trend_osaka = result.trend.dropna().reset_index()
trend_osaka.columns = ["Datum", "Trend"]
trend_osaka["City"] = "Osaka"

In [ ]:
trend_osaka

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Filtern: Nur Daten von 2015 bis 2024 behalten
df_delhi_filtered = df_delhi[(df_delhi.index.year >= 2015) & (df_delhi.index.year <= 2024)]

# Überprüfen, welche Jahre jetzt enthalten sind
print("Enthaltene Jahre:", sorted(df_delhi_filtered.index.year.unique()))

# Aggregiere die PM2.5-Daten auf monatlicher Basis (mittlere Werte)
df_delhi_monthly = df_delhi_filtered['Pm25'].resample('M').mean().dropna()

# Zeitreihenzerlegung (additives Modell mit period=12 für monatliche Daten
result = seasonal_decompose(df_delhi_monthly, model='additive', period=12)

# Visualisierung der Zerlegung
result.plot()

plt.suptitle('Zeitreihenzerlegung der monatlichen PM2.5-Daten in Delhi (2015-2024)', fontsize=16)
plt.tight_layout();

In [ ]:
# Trend extrahieren und als DataFrame speichern
trend_delhi = result.trend.dropna().reset_index()
trend_delhi.columns = ["Datum", "Trend"]
trend_delhi["City"] = "Delhi"

In [ ]:
trend_delhi

In [ ]:
trend_gesamt = pd.concat([trend_tokyo, trend_osaka, trend_delhi], ignore_index=True)
trend_gesamt.to_csv("data/trendlinien_pm25.csv", index=False)

In [ ]:
df_trend = pd.read_csv("data/trendlinien_pm25.csv")
print(df_trend["City"].unique())

# 1. Dokumentation

Was man später beim Skalieren nicht vergesen darf:

Liniendiagramm:

✅ Mein Plan für den skalierbaren Umgang mit mehr Städten:
🔹 1. Stadtanzahl begrenzen (z. B. Top 10 nach Jahresmittelwert)

# Top 10 Städte mit höchsten Mittelwerten im letzten Jahr
top_staedte = df_letztes_jahr.sort_values(by=auswahl, ascending=False).head(10)

So bleibt das Diagramm übersichtlich und zeigt die „interessantesten Fälle“ zuerst.

🔹 2. Farben automatisch generieren (mit Colormap)

Wir lassen Matplotlib die Farben aus einer Farbpalette wählen – z. B. "viridis", "plasma", "Set2", "tab10", etc.

import matplotlib.cm as cm
import numpy as np

# Colormap definieren
cmap = cm.get_cmap("tab10", len(top_staedte))

# Farben automatisch zuweisen
colors = [cmap(i) for i in range(len(top_staedte))]


Dann beim Plotten:

ax_bar.bar(
    top_staedte["City"],
    top_staedte[auswahl],
    color=colors
)

🔹 3. Optional: Dropdown mit Städteauswahl oder Jahr

So kann man z. B. sagen:

    „Zeig mir die Top 10 Städte im Jahr 2022“
    oder
    „Zeig mir die 5 Städte mit der niedrigsten NO₂-Belastung im aktuellen Jahr“

🎁 Fazit für später:

Wenn du den Datensatz erweiterst, dann machen wir:

    intelligente Auswahl, z. B. Top 10

    automatische Farben

    flexible Steuerung durch Dropdown oder Slider

So bleibt dein Dashboard schön, verständlich und performant – auch bei vielen Städten.

In [ ]:
# Skalieren der Trendlinien
# seasonal_decompose rechnet nicht so schnell, deshalb sollte man die Daten vorher abspeichern.

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Original-DataFrame mit vielen Städten
df = pd.read_csv("deine_komplettdaten.csv")
df["Datum"] = pd.to_datetime(df[["Year", "Month", "Day"]])
df = df.set_index("Datum")

alle_staedte = df["City"].unique()
alle_trends = []

for stadt in alle_staedte:
    df_stadt = df[df["City"] == stadt]["Pm25"].resample("M").mean().dropna()

    if len(df_stadt) < 24:  # Sicherstellen: genug Daten für Trendberechnung
        continue

    try:
        result = seasonal_decompose(df_stadt, model="additive", period=12)
        trend = result.trend.dropna().reset_index()
        trend.columns = ["Datum", "Trend"]
        trend["City"] = stadt
        alle_trends.append(trend)
    except:
        print(f"Fehler bei Stadt: {stadt}")
        continue

# Alles zusammenführen
trend_gesamt = pd.concat(alle_trends, ignore_index=True)

# Speichern
trend_gesamt.to_csv("data/trendlinien_pm25_alle_staedte.csv", index=False)
